In [48]:
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
from dateutil import relativedelta
import numpy as np
from IPython.display import display

In [50]:
personal_path = os.environ['USERPROFILE']
if personal_path == 'C:\\Users\\dinhhoang.nguyen.CONCENTRIX':
    middle_path = r'OneDrive - Concentrix Corporation\WFM-internal'
else:
    middle_path = r'Concentrix Corporation\Dinh Hoang Nguyen - WFM-internal'
user_credential = os.path.join(os.environ['USERPROFILE'], middle_path)

C:\Users\dinhhoang.nguyen.CONCENTRIX\OneDrive - Concentrix Corporation\WFM-internal


In [51]:
time_reset = dt(year=2023, month=8, day=1)
update_less = True

In [52]:
# IMPORT MASTER ROSTER
masterroster = pd.read_json(os.path.join(user_credential, r'DB\filejson\masterroster.json'))
masterroster['Employee_ID'] = masterroster['Employee_ID'].astype("Int64")

In [53]:
# IMPORT EPS
eps = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\eps_full.json'))
if update_less is True:
    eps = pd.read_json(os.path.join(user_credential, r'DB\filejson\eps.json'))
else:
    eps = pd.read_json(os.path.join(user_credential, r'DB\filejson\eps_full.json'))

In [54]:
#IMPORT CPI
if update_less is True:
    cpi = pd.read_json(os.path.join(user_credential, r'DB\filejson\cpi.json'))
else:
    cpi = pd.read_json(os.path.join(user_credential, r'DB\filejson\cpi_full.json'))

In [55]:
#IMPORT AHT
if update_less is True:
    aht = pd.read_json(os.path.join(user_credential, r'DB\filejson\aht.json'))
else:
    aht = pd.read_json(os.path.join(user_credential, r'DB\filejson\aht_full.json'))

In [56]:
# IMPORT SCHEDULE
if update_less is True:
    schedule = pd.read_json(os.path.join(user_credential, r'DB\filejson\schedule.json'))
else:
    schedule = pd.read_json(os.path.join(user_credential, r'DB\filejson\schedule_full.json'))

In [57]:
# IMPORT CUIC
if update_less is True:
    cuic = pd.read_json(os.path.join(user_credential, r'DB\filejson\cuic.json'))
else:
    cuic = pd.read_json(os.path.join(user_credential, r'DB\filejson\cuic_full.json'))

In [58]:
# IMPORT TL
tl = pd.read_json(os.path.join(user_credential, r'DB\filejson\tl.json'))

In [59]:
# quality
if update_less == True:
    quality = pd.read_json(os.path.join(user_credential, r'DB\filejson\quality.json'))
else:
    quality = pd.read_json(os.path.join(user_credential, r'DB\filejson\quality_full.json'))

In [60]:
# Frame
main_frame = schedule

## add Week
main_frame['Date'] = pd.to_datetime(main_frame['Date'], errors='coerce')
main_frame['Week'] = main_frame['Date'].dt.strftime('%Y%W')

In [61]:
# Frame
main_frame = schedule

## add Week
main_frame['Date'] = pd.to_datetime(main_frame['Date'], errors='coerce')
main_frame['Week'] = main_frame['Date'].dt.strftime('%Y%W')
main_frame['Date'] = main_frame['Date'].dt.date

## Add LOB Group
lob_map = {'EN': 'English', 'VICSP': 'Vietnamese CSP', 'VICSG': 'Vietnamese CSG', 'Senior VICSP': 'Senior CSP', 'NL': 'Unbabel', 'ID4': 'Unbabel', 'HE4': 'Unbabel', 'XT4': 'Unbabel', 'EL': 'Unbabel', 'TR': 'Unbabel', 'KO': 'Unbabel', 'UB CS': 'Unbabel', 'HU': 'Unbabel', 'FR': 'Unbabel', 'ZH': 'Unbabel', 'RU': 'Unbabel', 'UB PL': 'Unbabel', 'PT': 'Unbabel'}
## add tenure
main_frame['LOB Group'] = main_frame['LOB'].map(lob_map)

main_frame = pd.merge(main_frame, masterroster[['Employee_ID', 'Booking Login ID', 'TED Name', 'CUIC Name', 'PST_Start_Date', 'Wave #', 'Role']], left_on='Emp ID', right_on='Employee_ID', how='left')
main_frame['PST_Start_Date'] = pd.to_datetime(main_frame['PST_Start_Date'], format='mixed').dt.date
def TN(x):
    if x['PST_Start_Date'] is pd.NaT:
        return 'unknown'
    else:
        if x['Date'] - x['PST_Start_Date'] >= pd.Timedelta(days=90):
            return "TN"
        else:
            return "NH"
    
def tenure_days(x):
    if x['PST_Start_Date'] is pd.NaT:
        return 'unknown'
    else:
        days = x['Date'] - x['PST_Start_Date']
        if days <= pd.Timedelta(days=30):
            return '0-30'
        elif days <= pd.Timedelta(days=60):
            return '31-60'
        elif days <= pd.Timedelta(days=90):
            return '61-90'
        elif days <= pd.Timedelta(days=120):
            return '91-120'
        else:
            return '120+'
    
main_frame['Tenure'] = main_frame.apply(TN, axis=1)
main_frame['Tenure days'] = main_frame.apply(tenure_days, axis=1)
main_frame = main_frame.drop(columns=['Employee_ID'])
## add supervisor
main_frame = pd.merge(main_frame, tl[['Emp ID', 'Supervisor']], on='Emp ID', how='left')

In [62]:
## cpi processing
aht_cpi = cpi

## add phone cases
aht_cpi['#phone'] = aht_cpi.apply(lambda x: x['Number of Records'] if x['Channel'] == 'phone' else 0, axis=1)
aht_cpi['#non-phone'] = aht_cpi.apply(lambda x: x['Number of Records'] if x['Channel'] != 'phone' else 0, axis=1)

aht_cpi = aht_cpi.groupby(['Employee_ID', 'Date'], as_index=False)[['#phone', '#non-phone']].sum()
aht_cpi['total_cases'] = aht_cpi['#phone'] + aht_cpi['#non-phone']
### add to main frame
aht_cpi['Date'] = aht_cpi['Date'].dt.date
main_frame = pd.merge(main_frame, aht_cpi, left_on=['Emp ID', 'Date'], right_on=['Employee_ID', 'Date'], how='left')
main_frame = main_frame.drop(columns={'Employee_ID'})
main_frame[['#phone', '#non-phone', 'total_cases']] = main_frame[['#phone', '#non-phone', 'total_cases']].fillna(0)

In [63]:
## AHT
aht_aht = aht.drop(columns=['Start Time', 'End Time', '"Handling Time"', 'First Email Id'])

## add Eliminate Phone
def eliminate_phone(x):
    if x['Item Channel'] =='Phone' and x['Topic'] =='Unknown':
        return "Notphone"
    else:
        return x['Item Channel']
aht_aht['Eliminate Phone'] = aht_aht.apply(eliminate_phone, axis=1)

aht_mr = masterroster[['Employee_ID', 'PST_Start_Date', 'CUIC Name', 'Full name', 'Role', 'TED Name', 'Wave #']]

aht_aht = pd.merge(aht_aht, aht_mr, left_on='Staff', right_on='TED Name', how='left')

aht_aht = pd.merge(aht_aht, tl, left_on ='Employee_ID_x', right_on = 'Emp ID', how='left')
aht_lob = schedule[['Emp ID', 'Date', 'LOB']]
aht_lob['Date'] = pd.to_datetime(aht_lob['Date'], format ='mixed')
aht_aht = pd.merge(aht_aht, aht_lob, left_on = ['Employee_ID_x', 'Date'], right_on=['Emp ID', 'Date'], how='left')
aht_aht = aht_aht.drop(columns=['Employee_ID_y', 'Emp ID_x', 'Emp ID_y'])
aht_aht = aht_aht.rename(columns={'Employee_ID_x':'Employee_ID'})
#Add talk time
def talk_time(x):
    try:
        from_left = 11
        to_right = str(x['Tooltip Phone Time']).find(' Wrap Time')
        return int(x['Tooltip Phone Time'][from_left:to_right])
    except:
        return 0
def wrap_time(x):
    try:
        from_left = int(str(x['Tooltip Phone Time']).find('Wrap Time'))+11
        to_right = str(x['Tooltip Phone Time']).find(' Hold Time')
        return int(x['Tooltip Phone Time'][from_left:to_right])
    except:
        return 0
def hold_time(x):
    try:
        from_left = int(str(x['Tooltip Phone Time']).find('Hold Time'))+10
        return int(x['Tooltip Phone Time'][from_left:])
    except:
        return 0    
aht_aht['Talk Time'] = aht_aht.apply(talk_time, axis=1)
aht_aht['Wrap Time'] = aht_aht.apply(wrap_time, axis=1)
aht_aht['Hold Time'] = aht_aht.apply(hold_time, axis=1)

def act_handling_time(x):
    if x['Eliminate Phone'] =='Phone':
        return x['Talk Time'] + x['Wrap Time'] + x['Hold Time']
    else:
        return x['Handling Time']

# add realchannel
def realchannel(x):
    if x['Item Channel'] == 'Phone' and x['Topic'] == 'Unknown':
        return "Phone1"
    else:
        return x['Item Channel']
aht_aht['Real Channel'] = aht_aht.apply(realchannel, axis=1)

# add total inbound
def totalinbound(x):
    if x['Item Channel'] in ['Email', 'Live Chat', 'Messaging', 'Unknown']:
        return x['Handling Time']
    else:
        return x['Talk Time'] + x['Wrap Time'] + x['Hold Time']

aht_aht['Total Inbound'] = aht_aht.apply(totalinbound, axis=1)

#add Act handling time
aht_aht['Act Handling Time'] = aht_aht.apply(act_handling_time, axis=1)
#Date to_datetime
aht_aht['Date'] = pd.to_datetime(aht_aht['Date'], format='mixed').dt.strftime('%Y-%m-%d')

C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_14348\1391897611.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aht_lob['Date'] = pd.to_datetime(aht_lob['Date'], format ='mixed')


In [64]:
# time & count for AHT
aht_aht['AHT Phone time'] = aht_aht.apply(lambda x: x['Act Handling Time'] if x['Eliminate Phone'] == 'Phone' else 0, axis=1)
aht_aht['AHT Phone count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] == 'Phone' else 0, axis=1)

aht_aht['AHT Non-phone time'] = aht_aht.apply(lambda x: x['Act Handling Time'] if x['Eliminate Phone'] in ['Email', 'Unknown', 'Messaging', 'SNR', 'BH'] else 0, axis=1)
aht_aht['AHT Non-phone count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] in ['Email', 'Unknown', 'Messaging', 'SNR', 'BH'] else 0, axis=1)

aht_aht['Overall AHT time'] = aht_aht.apply(lambda x: x['Act Handling Time'] if x['Eliminate Phone'] != 'Notphone' else 0, axis=1)
aht_aht['Overall AHT count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] != 'Notphone' else 0, axis=1)

aht_aht['Hold (phone) time'] = aht_aht.apply(lambda x: x['Hold Time'] if x['Eliminate Phone'] == 'Phone' else 0, axis=1)
aht_aht['Hold (phone) count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] == 'Phone' else 0, axis=1)

aht_aht['AACW (phone) time'] = aht_aht.apply(lambda x: x['Wrap Time'] if x['Eliminate Phone'] == 'Phone' else 0, axis=1)
aht_aht['AACW (phone) count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] == 'Phone' else 0, axis=1)

aht_aht['Avg Talk Time time'] = aht_aht.apply(lambda x: x['Talk Time'] if x['Eliminate Phone'] == 'Phone' else 0, axis=1)
aht_aht['Avg Talk Time count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] == 'Phone' else 0, axis=1)

In [65]:
# group by AHT
aht_aht = aht_aht.groupby(['Date', 'Employee_ID'], as_index=False)[['AHT Phone time', 'AHT Phone count', 'AHT Non-phone time', 'AHT Non-phone count', 'Overall AHT time', 'Overall AHT count', 'Hold (phone) time', 'Hold (phone) count', 'AACW (phone) time', 'AACW (phone) count', 'Avg Talk Time time', 'Avg Talk Time count']].sum()

In [66]:
# add aht_aht to main frame
main_frame['Date'] = pd.to_datetime(main_frame['Date'], format='mixed').dt.strftime('%Y-%m-%d')
main_frame = pd.merge(main_frame, aht_aht, left_on =['Emp ID', 'Date'], right_on=['Employee_ID', 'Date'], how='left')
main_frame = main_frame.drop(columns={'Employee_ID'})
main_frame['AHT Phone time'] = main_frame['AHT Phone time'].fillna(0)
main_frame['AHT Phone count'] = main_frame['AHT Phone count'].fillna(0)

main_frame['AHT Non-phone time'] = main_frame['AHT Non-phone time'].fillna(0)
main_frame['AHT Non-phone count'] = main_frame['AHT Non-phone count'].fillna(0)

main_frame['Overall AHT time'] = main_frame['Overall AHT time'].fillna(0)
main_frame['Overall AHT count'] = main_frame['Overall AHT count'].fillna(0)

main_frame['Hold (phone) time'] = main_frame['Hold (phone) time'].fillna(0)
main_frame['Hold (phone) count'] = main_frame['Hold (phone) count'].fillna(0)

main_frame['AACW (phone) time'] = main_frame['AACW (phone) time'].fillna(0)
main_frame['AACW (phone) count'] = main_frame['AACW (phone) count'].fillna(0)

main_frame['Avg Talk Time time'] = main_frame['Avg Talk Time time'].fillna(0)
main_frame['Avg Talk Time count'] = main_frame['Avg Talk Time count'].fillna(0)

In [67]:
# cuic
aht_cuic = cuic
aht_cuic = aht_cuic.groupby(['Employee_ID', 'Session Date'], as_index=False)[['AgentAvailTime']].sum()
aht_cuic['AgentAvailTime']=aht_cuic['AgentAvailTime']*24
aht_cuic['Session Date'] = pd.to_datetime(aht_cuic['Session Date'], format='mixed').dt.strftime('%Y-%m-%d')

In [68]:
# add cuic to main frame
main_frame = pd.merge(main_frame, aht_cuic, left_on =['Emp ID', 'Date'], right_on=['Employee_ID', 'Session Date'], how='left')
main_frame = main_frame.drop(columns={'Employee_ID', 'Session Date'})
main_frame['AgentAvailTime'] = main_frame['AgentAvailTime'].fillna(0)

In [69]:
# eps
aht_eps = eps
# calculate time
aht_eps['phone_time'] = aht_eps.apply(lambda x: x['Total Time'] if x['BPE Code'] in ['Ready or Talking','Finalize Call'] else 0, axis=1)/3600
aht_eps['nonphone_time'] = aht_eps.apply(lambda x: x['Total Time'] if x['BPE Code'] == 'Picklist - off Phone' else 0, axis=1)/3600
aht_eps['productive_time'] = aht_eps.apply(lambda x: x['Total Time'] if x['BPE Code'] in ['Picklist - off Phone', 'Ready or Talking''Finalize Call','RONA','Unscheduled Picklist','Payment Processing','Social Media','Mass Issue','Project'] else 0, axis=1)/3600

## add session date
aht_eps['Session login date'] = pd.to_datetime(aht_eps['Session login date'], format='mixed').dt.date
aht_eps['Session login date-1'] = aht_eps['Session login date'] - pd.Timedelta(days=1)
aht_eps = pd.merge(aht_eps, schedule[['Emp ID', 'Date', 'Shift_type']], left_on=['EID', 'Session login date-1'], right_on=['Emp ID', 'Date'], how='left')

def session_date(x):
    if x['Shift_type'] == 'NS' and x['Session login time'] <= t(12, 0, 0):
        return x['Session login date-1']
    else:
        return x['Session login date']
    
aht_eps['Session login time'] = pd.to_datetime(aht_eps['Session login time'], format='mixed').dt.time
aht_eps['Session date'] = aht_eps.apply(session_date, axis=1)

aht_eps = aht_eps.groupby(['EID', 'Session date'], as_index=False)[['phone_time', 'nonphone_time', 'productive_time']].sum()
aht_eps = aht_eps.rename(columns={'Session date': 'Date'})

aht_eps['Date'] = pd.to_datetime(aht_eps['Date'], format = 'mixed').dt.strftime('%Y-%m-%d')

In [70]:
## add aht_eps to main frame
main_frame = pd.merge(main_frame, aht_eps, left_on =['Emp ID', 'Date'], right_on=['EID', 'Date'], how='left')
main_frame = main_frame.drop(columns={'EID'})
main_frame[['phone_time', 'nonphone_time', 'productive_time']] = main_frame[['phone_time', 'nonphone_time', 'productive_time']].fillna(0)

In [71]:
main_frame['Date'] = pd.to_datetime(main_frame['Date'], format='mixed').dt.strftime('%Y-%m-%d')

In [72]:
quality = quality.rename(columns={' score_question_weight': 'score_question_weight'})
quality[['score_n', 'score_question_weight']] = quality[['score_n', 'score_question_weight']].fillna(0)

quality['customer_score'] = quality.apply(lambda x: x['score_n'] if x['sections'] == 'CUSTOMER' else 0, axis=1)
quality['customer_weight'] = quality.apply(lambda x: x['score_question_weight'] if x['sections'] == 'CUSTOMER' else 0, axis=1)

quality['business_score'] = quality.apply(lambda x: x['score_n'] if x['sections'] == 'BUSINESS' else 0, axis=1)
quality['business_weight'] = quality.apply(lambda x: x['score_question_weight'] if x['sections'] == 'BUSINESS' else 0, axis=1)

quality['compliance_score'] = quality.apply(lambda x: x['score_n'] if x['sections'] == 'COMPLIANCE' else 0, axis=1)
quality['compliance_weight'] = quality.apply(lambda x: x['score_question_weight'] if x['sections'] == 'COMPLIANCE' else 0, axis=1)

In [73]:
# group by quality
quality = quality.groupby(['agent_username', 'eval_date'], as_index=False)[['customer_score', 'customer_weight', 'business_score', 'business_weight', 'compliance_score', 'compliance_weight']].sum()

In [74]:
# add quality to main frame

main_frame = pd.merge(main_frame, quality, left_on=['Booking Login ID', 'Date'], right_on=['agent_username', 'eval_date'], how='left')
main_frame[['customer_score', 'customer_weight', 'business_score', 'business_weight', 'compliance_score', 'compliance_weight']] = main_frame[['customer_score', 'customer_weight', 'business_score', 'business_weight', 'compliance_score', 'compliance_weight']].fillna(0)
main_frame = main_frame.drop(columns=['agent_username', 'eval_date'])

In [75]:
# change datatype mainframe['Date'] to json
main_frame['Date'] = pd.to_datetime(main_frame['Date'], format='mixed').dt.strftime('%Y-%m-%d')

In [76]:
# CSAT
if update_less == True:
    csat = pd.read_json(os.path.join(user_credential, r'DB\filejson\csat.json'))
else:
    csat = pd.read_json(os.path.join(user_credential, r'DB\filejson\csat_full.json'))
    
csat['Date'] = csat['Date'].dt.date
## get LOB
csat = pd.merge(csat, schedule[['Emp ID', 'Date', 'LOB']], left_on=['Employee_ID', 'Date'], right_on=['Emp ID', 'Date'], how='left')
csat = csat.drop(columns={'Emp ID'})
csat = csat.dropna(subset=['LOB'])

# count phone survey
def phone_surveycount_overall(x):
    if x['Channel'] == 'phone':
        if x['LOB'] != 'VICSG':
            return 1
        else:
            if x['Language'] == 'Vietnamese':
                return 1
            else:
                return 0
    else:
        return 0
# count phone csat
def phone_csatcount_overall(x):
    if x['Channel'] == 'phone':
        if x['Csat 2.0 Score'] in ['Satisfied', 'Very Satisfied']:
            if x['LOB'] != 'VICSG':
                return 1
            else:
                if x['Language'] == 'Vietnamese':
                    return 1
                else:
                    return 0
        else:
            return 0
    else:
        return 0

# count non phone survey
def nonphone_surveycount_overall(x):
    if x['Channel'] != 'phone':
        if x['LOB'] != 'VICSG':
            return 1
        else:
            if x['Language'] == 'Vietnamese':
                return 1
            else:
                return 0
    else:
        return 0
    
def nonphone_csatcount_overall(x):
    if x['Channel'] != 'phone':
        if x['Csat 2.0 Score'] in ['Satisfied', 'Very Satisfied']:
            if x['LOB'] != 'VICSG':
                return 1
            else:
                if x['Language'] == 'Vietnamese':
                    return 1
                else:
                    return 0
        else:
            return 0
    else:
        return 0
        
csat['phone_survey'] = csat.apply(phone_surveycount_overall, axis=1)
csat['nonphone_survey'] = csat.apply(nonphone_surveycount_overall, axis=1)
csat['phone_csat'] = csat.apply(phone_csatcount_overall, axis=1)
csat['nonphone_csat'] = csat.apply(nonphone_csatcount_overall, axis=1)
csat['Date'] = pd.to_datetime(csat['Date'], format='mixed').dt.strftime('%Y-%m-%d')


In [77]:
# summarise csat
csat = csat.groupby(['Employee_ID', 'Date'], as_index=False)[['phone_survey', 'nonphone_survey', 'phone_csat', 'nonphone_csat']].sum()

In [78]:
# add csat to main frame
main_frame = pd.merge(main_frame, csat, left_on=['Emp ID', 'Date'], right_on=['Employee_ID', 'Date'], how='left')

In [79]:
# CSAT_reso Reso
csat_reso = pd.read_json(os.path.join(user_credential, r'DB\filejson\csat_reso_full.json'))
csat_reso['Date'] = csat_reso['Date'].dt.date

## get LOB
csat_reso = pd.merge(csat_reso, schedule[['Emp ID', 'Date', 'LOB']], left_on=['Employee_ID', 'Date'], right_on=['Emp ID', 'Date'], how='left')
csat_reso = csat_reso.drop(columns={'Emp ID'})
csat_reso = csat_reso.dropna(subset=['LOB'])

# count phone survey
def phone_surveycount_overall(x):
    if x['Channel'] == 'phone':
        if x['LOB'] != 'VICSG':
            return 1
        else:
            if x['Language'] == 'Vietnamese':
                return 1
            else:
                return 0
    else:
        return 0
# count phone csat_reso
def phone_csat_resocount_overall(x):
    if x['Channel'] == 'phone':
        if x['Csat 2.0 Score'] in ['Satisfied', 'Very Satisfied']:
            if x['LOB'] != 'VICSG':
                return 1
            else:
                if x['Language'] == 'Vietnamese':
                    return 1
                else:
                    return 0
        else:
            return 0
    else:
        return 0

# count non phone survey
def nonphone_surveycount_overall(x):
    if x['Channel'] != 'phone':
        if x['LOB'] != 'VICSG':
            return 1
        else:
            if x['Language'] == 'Vietnamese':
                return 1
            else:
                return 0
    else:
        return 0
    
def nonphone_csat_resocount_overall(x):
    if x['Channel'] != 'phone':
        if x['Csat 2.0 Score'] in ['Satisfied', 'Very Satisfied']:
            if x['LOB'] != 'VICSG':
                return 1
            else:
                if x['Language'] == 'Vietnamese':
                    return 1
                else:
                    return 0
        else:
            return 0
    else:
        return 0
        
csat_reso['phone_survey_reso'] = csat_reso.apply(phone_surveycount_overall, axis=1)
csat_reso['nonphone_survey_reso'] = csat_reso.apply(nonphone_surveycount_overall, axis=1)
csat_reso['phone_csat_reso'] = csat_reso.apply(phone_csat_resocount_overall, axis=1)
csat_reso['nonphone_csat_reso'] = csat_reso.apply(nonphone_csat_resocount_overall, axis=1)
csat_reso['Date'] = pd.to_datetime(csat_reso['Date'], format='mixed').dt.strftime('%Y-%m-%d')

# summarise csat_reso
csat_reso = csat_reso.groupby(['Employee_ID', 'Date'], as_index=False)[['phone_survey_reso', 'nonphone_survey_reso', 'phone_csat_reso', 'nonphone_csat_reso']].sum()

# add csat_reso to main frame
main_frame = main_frame.drop(columns={'Employee_ID'})
main_frame = pd.merge(main_frame, csat_reso, left_on=['Emp ID', 'Date'], right_on=['Employee_ID', 'Date'], how='left')

In [80]:
#PSAT
psat = pd.read_json(os.path.join(user_credential, r'DB\filejson\psat.json'))

def survey_count(x):
    if x['Agent understood my question'] != 'No Answer':
        q1 = 1
    else:
        q1 = 0
    if x['Agent did everything possible to help me'] != 'No Answer':
        q2 = 1
    else:
        q2 = 0
    survey_count = q1 + q2
    return survey_count

def survey_count_vnese(x):
    if x['Agent understood my question'] != 'No Answer' and x['Language'] == 'Vietnamese':
        q1 = 1
    else:
        q1 = 0
    if x['Agent did everything possible to help me'] != 'No Answer' and x['Language'] == 'Vietnamese':
        q2 = 1
    else:
        q2 = 0
    survey_count = q1 + q2
    return survey_count

def survey_count_american(x):
    if x['Agent understood my question'] != 'No Answer' and x['Language'] == 'English (American)':
        q1 = 1
    else:
        q1 = 0
    if x['Agent did everything possible to help me'] != 'No Answer' and x['Language'] == 'English (American)':
        q2 = 1
    else:
        q2 = 0
    survey_count = q1 + q2
    return survey_count

def survey_count_britain(x):
    if x['Agent understood my question'] != 'No Answer' and x['Language'] == 'English (Great Britain)':
        q1 = 1
    else:
        q1 = 0
    if x['Agent did everything possible to help me'] != 'No Answer' and x['Language'] == 'English (Great Britain)':
        q2 = 1
    else:
        q2 = 0
    survey_count = q1 + q2
    return survey_count


def psat_count(x):
    if x['Agent understood my question'] in ['Satisfied', 'Very Satisfied']:
        q1 = 1
    else:
        q1 = 0
    if x['Agent did everything possible to help me'] in ['Satisfied', 'Very Satisfied']:
        q2 = 1
    else:
        q2 = 0
    psat_count = q1 + q2
    return psat_count

def psat_count_vnese(x):
    if x['Agent understood my question'] in ['Satisfied', 'Very Satisfied'] and x['Language'] == 'Vietnamese':
        q1 = 1
    else:
        q1 = 0
    if x['Agent did everything possible to help me'] in ['Satisfied', 'Very Satisfied'] and x['Language'] == 'Vietnamese':
        q2 = 1
    else:
        q2 = 0
    psat_count = q1 + q2
    return psat_count

def psat_count_american(x):
    if x['Agent understood my question'] in ['Satisfied', 'Very Satisfied'] and x['Language'] == 'English (American)':
        q1 = 1
    else:
        q1 = 0
    if x['Agent did everything possible to help me'] in ['Satisfied', 'Very Satisfied'] and x['Language'] == 'English (American)':
        q2 = 1
    else:
        q2 = 0
    psat_count = q1 + q2
    return psat_count

def psat_count_britain(x):
    if x['Agent understood my question'] in ['Satisfied', 'Very Satisfied'] and x['Language'] == 'English (Great Britain)':
        q1 = 1
    else:
        q1 = 0
    if x['Agent did everything possible to help me'] in ['Satisfied', 'Very Satisfied'] and x['Language'] == 'English (Great Britain)':
        q2 = 1
    else:
        q2 = 0
    psat_count = q1 + q2
    return psat_count

psat['psat_survey'] = psat.apply(survey_count, axis=1)
psat['psat_count'] = psat.apply(psat_count, axis=1)

psat['psat_survey_vnese'] = psat.apply(survey_count_vnese, axis=1)
psat['psat_count_vnese'] = psat.apply(psat_count_vnese, axis=1)

psat['psat_survey_american'] = psat.apply(survey_count_american, axis=1)
psat['psat_count_american'] = psat.apply(psat_count_american, axis=1)

psat['psat_survey_britain'] = psat.apply(survey_count_britain, axis=1)
psat['psat_count_britain'] = psat.apply(psat_count_britain, axis=1)
# add ID
psat = pd.merge(psat, masterroster[['Employee_ID', 'TED Name']], left_on='Staff Name', right_on='TED Name', how='left')
psat = psat.drop(columns=['TED Name'])
psat['Date'] = pd.to_datetime(psat['Date'], format='mixed').dt.strftime('%Y-%m-%d')

In [81]:
# summarise psat
psat = psat.groupby(['Employee_ID', 'Date'], as_index=False)[['psat_survey', 'psat_count', 'psat_survey_vnese', 'psat_count_vnese', 'psat_survey_american', 'psat_count_american', 'psat_survey_britain', 'psat_count_britain']].sum()

In [82]:
# add psat to main frame
main_frame = pd.merge(main_frame, psat, left_on=['Emp ID', 'Date'], right_on=['Employee_ID', 'Date'], how='left')

In [83]:
# remove abundant cols
main_frame = main_frame.drop(columns={'Employee_ID_x', 'Employee_ID_y'})
# fillna
main_frame[['psat_survey', 'psat_count', 'phone_survey', 'nonphone_survey', 'phone_csat', 'nonphone_csat']] = main_frame[['psat_survey', 'psat_count', 'phone_survey', 'nonphone_survey', 'phone_csat', 'nonphone_csat']].fillna(0)

In [84]:
# get kpi_target
kpi_target = pd.read_json(os.path.join(user_credential, r'DB\filejson\kpi_target.json'))
main_frame['Week'] = main_frame['Week'].astype('int64')
main_frame = pd.merge(main_frame, kpi_target, on=['Week', 'Tenure days', 'LOB Group'], how='left')

In [85]:
# get attendance
attendance = pd.read_json(os.path.join(user_credential, r'DB\filejson\booking_attendance.json'))
attendance['Date'] = pd.to_datetime(attendance['Date'], format='mixed').dt.strftime('%Y-%m-%d')
# add attendance to main frame
main_frame = pd.merge(main_frame, attendance[['Emp ID', 'Date', 'Actual Leave', 'Attendance', 'Total scheduled']], on=['Emp ID', 'Date'], how='left')

In [86]:
# Get RONA
rona = pd.read_json(os.path.join(user_credential, r'DB\filejson\rona.json'))
rona = rona.groupby(['Session Date', 'Employee_ID'], as_index=False)['RONA'].sum()
rona['Session Date'] = pd.to_datetime(rona['Session Date'], format='mixed').dt.strftime('%Y-%m-%d')

## Add Rona to main frame
main_frame = pd.merge(main_frame, rona, left_on=['Emp ID', 'Date'], right_on=['Employee_ID', 'Session Date'], how='left')
main_frame = main_frame.drop(columns={'Employee_ID', 'Session Date'})
main_frame['RONA'] = main_frame['RONA'].fillna(0)

In [87]:
main_frame['PST_Start_Date'] = pd.to_datetime(main_frame['PST_Start_Date'], format='mixed').dt.strftime('%Y-%m-%d')
main_frame['Date'] = pd.to_datetime(main_frame['Date'], format='mixed').dt.strftime('%Y-%m-%d')

In [ ]:
# Export to json
if update_less is False:
    main_frame.loc[(pd.to_datetime(main_frame['Date'], format='mixed')<time_reset)&(pd.to_datetime(main_frame['Date'], format='mixed')>dt(year=2023, month=1, day=1))].to_json(os.path.join(user_credential, r'DB\filejson\kpi_storage.json'), orient='records')
else:
    main_storage = pd.read_json(os.path.join(user_credential, r'DB\filejson\kpi_storage.json'))
    main_frame = pd.concat([main_storage, main_frame.loc[pd.to_datetime(main_frame['Date'], format='mixed')>=time_reset]])
    main_frame['Date'] = pd.to_datetime(main_frame['Date'], format='mixed').dt.strftime('%Y-%m-%d')
    # main_frame.to_json(os.path.join(user_credential, r'DB\filejson\kpi.json'), orient='records')
    main_frame.to_csv(os.path.join(user_credential, r'DB\filecsv\[BcomDB] KPI.csv'), index=False)
    main_frame.to_csv(os.path.join(personal_path, r'Concentrix Corporation\Tung Quan Le - BKN\AgentDetail\[BcomDB] KPI.csv'), index=False)
# main_frame.to_csv(os.path.join(r'C:', user_credential, r'DB\filecsv\kpi.csv'), index=False)

In [4]:
print("Ha Thanh Le Xinh Dep 10 dim")
print("Toi that pro")

Ha Thanh Le Xinh Dep
Toi that pro
